Tarefa 1: Extração de Dados

Escreva em Python os comandos para ligar à base de dados SQL e extrair dados das tabelas relevantes usando comandos SQL embebidos em Python. Use o código SQL adequado aos dados a extrair. Use libraries como SQLAlchemy e Pandas para interação com a base de dados.

In [ ]:
#Importar os módulos e funções necessárias

import pandas as pd
from sqlalchemy import create_engine
import getpass

In [ ]:
#Criação das variáveis necessárias para ligar ao servidor da base de dados de DDM

server = 'se-dbm-4b.database.windows.net'
database = 'dbm-4b'
username = 'g02'
password = getpass.getpass('Please provide the password for group 2')
driver = 'SQL Server'

In [ ]:
#Definição da connection string com as variáveis anteriormente criadas

connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'

In [ ]:
#Criação da variável engine com a connection à base de dados

engine = create_engine(connection_string)

In [ ]:
#Tabela + dataframe com a informação dos produtos e vendas

p_query = 'SELECT sod.productID, sod.orderqty, sod.unitprice, sod.unitpricediscount, sod.linetotal, p.name, p.standardcost, p.color, \
    soh.orderdate FROM SalesOrderDetail sod \
        LEFT JOIN product p ON p.ProductID = sod.ProductID \
            LEFT JOIN SalesOrderHeader soh ON soh.SalesOrderID = sod.SalesOrderID'

dfp = pd.read_sql(p_query, engine)

In [90]:
#Tabela + dataframe  com informação das categorias dos clientes e vendas

pc_query = 'SELECT c.CustomerID, c.CompanyName, c.FirstName, c.LastName, a.city, a.countryregion, soh.salesorderid, sod.linetotal \
        FROM Customer c \
            LEFT JOIN CustomerAddress ca on ca.customerID = c.CustomerID \
                LEFT JOIN Address a ON a.AddressID = ca.addressID \
                    LEFT JOIN SalesOrderHeader soh ON soh.CustomerID = c.CustomerID \
                        LEFT JOIN SalesOrderDetail sod ON sod.salesorderid = soh.salesorderid'

dfpc = pd.read_sql(pc_query, engine)

In [ ]:
engine.dispose()

Tarefa 2: Data Cleaning and Preprocessing

 Execute, se necessário, tarefas de limpeza de dados para lidar com quaisquer valores ausentes ou 
anomalias (ex. Tipos de dados inadequados) para garantir que os tipos de dados estejam corretos e 
adequados para análise.


Criámos funções de fácil uso para analisar as datagrames

In [ ]:
#Verificar se existem nulls e onde estão
def columns_with_nulls(df):
    return [col for col in df.columns if df[col].isnull().any()]

#Contar os nulls nas colunas

def columns_nulls_total(df):
    print(df.isnull().sum())

# Verificar os data types das colunas
def check_column_types(df):
    return df.dtypes


Analisar e tratar informação do Dataframe DFP

 - Analisar de forma geral toda a info do dataframe

In [89]:

print(dfp.describe())

        productID    orderqty    unitprice  unitpricediscount     linetotal  \
count  542.000000  542.000000   542.000000         542.000000    542.000000   
mean   882.732472    3.850554   382.870079           0.015498   1307.546408   
std     88.079302    3.297592   472.204091           0.073791   2348.085470   
min    707.000000    1.000000     1.374000           0.000000      2.994000   
25%    810.750000    2.000000    32.394000           0.000000     89.982000   
50%    893.000000    3.000000   149.874000           0.000000    316.860000   
75%    958.000000    5.000000   602.346000           0.000000   1390.266000   
max    999.000000   25.000000  1466.010000           0.400000  19136.137500   

       standardcost  
count    542.000000  
mean     385.514897  
std      473.079908  
min        0.856500  
25%       26.176300  
50%      144.593800  
75%      601.743700  
max     1554.947900  


In [94]:
print(dfp.head())

   productID  orderqty  unitprice  unitpricediscount  linetotal  \
0        836         1    356.898                0.0    356.898   
1        822         1    356.898                0.0    356.898   
2        907         1     63.900                0.0     63.900   
3        905         4    218.454                0.0    873.816   
4        983         2    461.694                0.0    923.388   

                               name  standardcost   color  
0      ML Road Frame-W - Yellow, 48      360.9428  Yellow  
1      ML Road Frame-W - Yellow, 38      360.9428  Yellow  
2                       Rear Brakes       47.2860  Silver  
3  ML Mountain Frame-W - Silver, 42      199.3757  Silver  
4         Mountain-400-W Silver, 46      419.7784  Silver  


 - Apagar coluna que não necessitamos

In [ ]:
dfp = dfp.drop(columns=['orderdate'])

- Descobrir nome das colunas com nulls

In [ ]:
print(columns_with_nulls(dfp))

- Verificar quantidade de nulls para ver a sua relevância

In [ ]:
print(columns_nulls_total(dfp))

- Alterar os nulls por outro valor na df


In [ ]:
#Existe 64 nulls em 'color' da df, como não sabemos a cor substituimos por 'Not_Defined'

dfp.fillna('NotDefined', inplace=True)

- Analisar datatypes de cada coluna - o tipo de dados está ok para nós

In [ ]:
print(check_column_types(dfpc))

- Mudar nomes de colunas da dataframe para algo mais legível

In [ ]:
dfp.rename(columns={'linetotal': 'totalsales'}, inplace=True)


Analisar e tratar informação do Dataframe DFPC

 - Analisar de forma geral toda a info do dataframe

In [92]:

print(dfpc.describe())

         CustomerID  salesorderid     linetotal
count   1367.000000    542.000000    542.000000
mean   20332.318947  71848.097786   1307.546408
std    13775.758629     60.577654   2348.085470
min        1.000000  71774.000000      2.994000
25%      539.500000  71784.000000     89.982000
50%    29660.000000  71845.000000    316.860000
75%    29929.000000  71902.000000   1390.266000
max    30118.000000  71946.000000  19136.137500


In [95]:
print(dfpc.head())

   CustomerID                 CompanyName FirstName    LastName  city  \
0           1                A Bike Store   Orlando         Gee  None   
1           2          Progressive Sports     Keith      Harris  None   
2           3    Advanced Bike Components     Donna    Carreras  None   
3           4       Modular Cycle Systems     Janet       Gates  None   
4           5  Metropolitan Sports Supply      Lucy  Harrington  None   

  countryregion  salesorderid  linetotal  
0          None           NaN        NaN  
1          None           NaN        NaN  
2          None           NaN        NaN  
3          None           NaN        NaN  
4          None           NaN        NaN  


 - Apagar coluna que não necessitamos

In [99]:
dfpc = dfpc.drop(columns=['FirstName','LastName'])

- Descobrir nome das colunas com nulls

In [100]:
print(columns_with_nulls(dfp))

[]


- Verificar quantidade de nulls para ver a sua relevância

In [101]:
print(columns_nulls_total(dfp))

productID            0
orderqty             0
unitprice            0
unitpricediscount    0
linetotal            0
name                 0
standardcost         0
color                0
dtype: int64
None


- Alterar "NaN" e "None" por outro valor na df


In [107]:
dfpc['linetotal'] = dfpc['linetotal'].replace('NaN',0)
dfpc['salesorderid'] = dfpc['salesorderid'].replace('NaN','no sales')
dfpc['city'] = dfpc['city'].replace('None','not defined')
dfpc['countryregion'] = dfpc['countryregion'].replace('None','not defined')

- Analisar datatypes de cada coluna - o tipo de dados está ok para nós

In [108]:
print(check_column_types(dfpc))

CustomerID         int64
CompanyName       object
city              object
countryregion     object
salesorderid     float64
linetotal        float64
dtype: object


- Mudar nomes de colunas da dataframe para algo mais legível

In [109]:
dfpc.rename(columns={'linetotal': 'totalsales'}, inplace=True)
dfpc.rename(columns={'countryregion': 'country'}, inplace=True)


Tarefa 3:  Exploratory Data Analysis (EDA)

 Calcule estatísticas básicas, como vendas totais, vendas médias por dia e vendas médias por categoria de produto.
 Identifique os produtos e categorias mais vendidos.
 Analise as tendências de vendas ao longo do tempo, como as tendências de vendas mensais.


- temos de ter análise para:
    - quantidade vendida por produto
    - valor vendido por produto
    - margem de cada produto (preço venda - custo)
    - vendas por company name
    - vendas por pais

É só colocar isto no excel e formatar as colunas para ficarem bonitas, aparentemente ele vai dar valor a isso

Temos de criar pelo menos 2 sheets, podemos ter duas com as análises acima, mas podiamos criar uma primeira, como ele sugere, com todas as alterações que fizemos acima

Tarefa 4: Grave os dados num ficheiro Excel

Guarde os dados resultantes das suas análises num ficheiro Excel com diferentes Sheets.

In [ ]:
#Definição do nome do meu ficheiro

my_Excel_file ='ProjetoPPL_Grupo2.xlsx'

In [ ]:
#Criação do ficheiro excel

my_writer = pd.ExcelWriter(my_Excel_file, datetime_format='YYYY-MM-DD')

In [ ]:
#Colocar as tablelas por sheets no excel

#dfp.to_excel(my_writer, sheet_name='First Query', index=False)

In [ ]:
#Fecha o ficheiro Excel

my_writer.close()